<a href="https://colab.research.google.com/github/pb-roshith/LSTM-sentiment-analysis-on-IMDB-review/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install kaggle

In [9]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other


In [11]:
from zipfile import ZipFile
dataset='/content/imdb-dataset-of-50k-movie-reviews.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print("extracted successfully dude")

extracted successfully dude


In [12]:
!ls

'IMDB Dataset.csv'			 kaggle.json
 imdb-dataset-of-50k-movie-reviews.zip	 sample_data


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [14]:
data = pd.read_csv('/content/IMDB Dataset.csv')

In [15]:
data.shape

(50000, 2)

In [16]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [18]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [19]:
data.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)

<ipython-input-19-87b5a72e3e4c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({"sentiment":{"positive":1, "negative":0}}, inplace=True)


In [20]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [21]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",0
49997,I am a Catholic taught in parochial elementary...,0
49998,I'm going to have to disagree with the previou...,0
49999,No one expects the Star Trek movies to be high...,0


In [22]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [23]:
train_data.shape

(40000, 2)

In [24]:
test_data.shape

(10000, 2)

In [25]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']), maxlen=200)

In [26]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [27]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [28]:
x_train[0]

array([1935,    1, 1200,    4, 2946, 3749, 1828,    2,  147,  144,    3,
        228,    4,    3,  207,  323,    2,  144, 1083,   16,   88,    4,
        132, 2871,   18,   10,  153,   99,    4,    1, 4020,  302,   11,
         17, 1001,   35,    1,  496,  492, 2619,  249,   71,   77,  107,
        107,  698,   60,   86, 1047, 1363,    5,  229,  132,   23, 4360,
         31,  138,  209, 1154,   14, 4501,   31,    3, 2386,    2,    8,
         11,    6,    3,  445,   14,  624,    4,    1,  718, 2959,    1,
       1278,    2,   71, 3616,    1,  166, 1507,    1, 1245,    5, 1629,
          1,  879, 1268,    5,    1,  310,  140, 2894,    2,  410,  633,
          7,    7,    1,  269,    6, 3553, 1000,    5,   26,   39,   14,
       1381,  217,   65,    2,   46,    6,   30,  219,   27,  193, 1484,
          8, 1101,   18,   10, 4905,   84,    1,  226,   66,  356,   68,
         54,   27,    5, 3600,   15,   44,   21,  192,    5,    3,  889,
       3511, 1758,   22,   25,    5,  158,  196,  1

In [30]:
y_train = train_data['sentiment']
y_test = test_data['sentiment']

In [31]:
y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


In [32]:
y_test

,sentiment
33553,1
9427,1
199,0
12447,1
39489,0
...,...
28567,0
25079,1
18707,1
15200,0


In [36]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=32, input_length=200))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [38]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [40]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 311s 623ms/step - accuracy: 0.8601 - loss: 0.3410 - val_accuracy: 0.8673 - val_loss: 0.3132
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 318s 614ms/step - accuracy: 0.7714 - loss: 0.4870 - val_accuracy: 0.8269 - val_loss: 0.4066
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 307s 584ms/step - accuracy: 0.8495 - loss: 0.3585 - val_accuracy: 0.8630 - val_loss: 0.3581
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 331s 600ms/step - accuracy: 0.8823 - loss: 0.2904 - val_accuracy: 0.8740 - val_loss: 0.3207
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 309s 617ms/step - accuracy: 0.8970 - loss: 0.2616 - val_accuracy: 0.8823 - val_loss: 0.2892


In [41]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 28s 86ms/step - accuracy: 0.8840 - loss: 0.2862
Test Loss: 0.2817074954509735, Test Accuracy: 0.8863000273704529


In [42]:
def pred(user_query):
  sequence = tokenizer.texts_to_sequences([user_query])
  padded_sequence = pad_sequences(sequence, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction > 0.5 else "negative"
  return sentiment

In [43]:
pred("This movie is amazing! I loved it.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 556ms/step


'positive'

In [44]:
pred("I hate this movie.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


'negative'

In [46]:
model.save('LSTM_movie_review.keras')

In [47]:
from tensorflow.keras.models import load_model

In [48]:
model = load_model('LSTM_movie_review.keras')

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [62]:
review = "very good movie."
x_new = tokenizer.texts_to_sequences([review])  # <- put inside list []
x_new = pad_sequences(x_new, maxlen=200)
predictions = model.predict(x_new)
sentiment1 = "positive" if predictions[0] > 0.5 else "negative"
print(sentiment1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 277ms/step
positive


In [63]:
review = "very bad movie."
x_new = tokenizer.texts_to_sequences([review])  # <- put inside list []
x_new = pad_sequences(x_new, maxlen=200)
predictions = model.predict(x_new)
sentiment1 = "positive" if predictions[0] > 0.5 else "negative"
print(sentiment1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
negative
